In [1]:
import os
import sys
import time
import yaml
import torch
import pandas as pd
import numpy as np 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from pytorch_lightning import LightningModule
from torch_geometric.nn import aggr
from torch.utils.checkpoint import checkpoint
import torch.optim as optim
from torch_scatter import scatter_add
from torch import Tensor
from torch_geometric.nn import MessagePassing
from typing import List, Optional
from sklearn.metrics import roc_auc_score
import warnings 
from torch_geometric.utils import to_networkx
from torch.nn import Sequential as Seq, Linear, ReLU, Sigmoid
from torch.optim.lr_scheduler import StepLR
from collections import namedtuple

from pytorch_lightning import LightningModule
import pytorch_lightning as pl
import pennylane as qml

TORCH = torch.__version__
CUDA = torch.version.cuda

print(f"Formatted PyTorch Version: {TORCH}")
print(f"Formatted CUDA Version: {CUDA}")

/hpc/m3/python/3.11.11/tensorflow-2025.08.21/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Formatted PyTorch Version: 2.9.1+cu128
Formatted CUDA Version: 12.8


In [2]:
class GraphDataset(Dataset):
    def __init__(self, home_dir, sub_dir, preprocess=True, hparams=None):
        self.base_path = os.path.join(home_dir, sub_dir)
        self.file_names = self._get_file_names()
        self.preprocess = preprocess
        self.hparams = hparams if hparams is not None else {}

    def _get_file_names(self):
        file_names = []
        for file_name in os.listdir(self.base_path):
            if file_name.endswith('.pyg'):  # Adjust this condition as needed
                file_names.append(file_name)
        return file_names

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = os.path.join(self.base_path, file_name)
        #data = torch.load(file_path)
        data = torch.load(file_path, weights_only=False)

        print(f"Loaded data from {file_path}")

        # Remove 'scores' from data if it exists
        if 'scores' in data:
            del data['scores']

        
        if self.preprocess:
            data = self.preprocess_event(data)
        return data

    def __len__(self):
        return len(self.file_names)

    def preprocess_event(self, data):
        print("Preprocessing data")
        data = self.add_edge_features(data)
        return data

    def add_edge_features(self, data):
        edge_features = self.hparams.get("edge_features", [])
        data = handle_edge_features(data, edge_features)
        return data

def handle_edge_features(data, edge_features):
    src, dst = data.edge_index
    
    if "dr" in edge_features and not ("dr" in data.keys()):
        data.dr = data.r[dst] - data.r[src]

    if "dz" in edge_features and not ("dz" in data.keys()):
        data.dz = data.z[dst] - data.z[src]

    if "dphi" in edge_features and not ("dphi" in data.keys()):
        dphi = data.phi[dst] - data.phi[src]
        data.dphi = torch.atan2(torch.sin(dphi), torch.cos(dphi))

    
    return data

In [3]:
hparams = {
    "stage": "edge_classifier",
    "model": "QuantumInteractionGNN2",
    "input_dir": "/users/santoshp/uiuc/quantumGNN/data/",
    "stage_dir": "/users/santoshp/uiuc/quantumGNN/model/",
    "project": "CTD_Quantum_2025",
    "accelerator": "gpu",
    "devices": 1,
    "nodes": 1,

    # Dataset parameters
    "data_split": [10, 5, 5],
    "dataset_class": "GraphDataset",
    "undirected": False,

    # Weighting and truth parameters
    "weighting": [
        {
            "weight": 1.0,
            "conditions": {
                "y": True,
                "pt": [1000.0, float("inf")],
                "nhits": [3, float("inf")],
            },
        }
    ],

    "hard_cuts": {
        "pt": [1000.0, float("inf")],
    },

    # Model parameters
    "edge_cut": 0.5,
    "node_features": ["r", "phi", "z"],
    "node_scales": [1000.0, 3.14159265359, 1000.0],
    "edge_features": ["dr", "dphi", "dz"],

    # Quantum-specific parameters
    "n_qubits": 2,
    "n_qlayers": 1,
    "quantum_device": "default.qubit",

    # Network architecture
    "hidden": 16,
    "n_graph_iters": 8,
    "n_node_encoder_layers": 1,
    "n_edge_encoder_layers": 1,
    "n_node_net_layers": 3,
    "n_edge_net_layers": 3,
    "n_node_decoder_layers": 1,
    "n_edge_decoder_layers": 1,

    # Normalization and activations
    "layernorm": False,
    "output_layer_norm": False,
    "edge_output_transform_final_layer_norm": False,
    "batchnorm": True,
    "output_batch_norm": False,
    "edge_output_transform_final_batch_norm": False,
    "bn_track_running_stats": False,
    "hidden_activation": "ReLU",
    "output_activation": "ReLU",
    "edge_output_transform_final_activation": None,

    # Message passing options
    "concat": True,
    "node_net_recurrent": True,
    "edge_net_recurrent": True,
    "in_out_diff_agg": True,
    "checkpointing": True,

    # Training parameters
    "warmup": 5,
    "lr": 0.0001,
    "min_lr": 0.00005,
    "factor": 0.9,
    "patience": 15,
    "max_epochs": 100,
    "max_training_graph_size": 2800000,
    "debug": False,
    "num_workers": [8, 8, 8],
}


In [4]:
home_dir = '/users/santoshp/uiuc/quantumGNN/data/'

test = 'test_set/'
train = 'train_set/'
val = 'val_set/'


test_dataset = GraphDataset(home_dir, test, preprocess=True, hparams=hparams)
train_dataset = GraphDataset(home_dir, train, preprocess=True, hparams=hparams)
val_dataset = GraphDataset(home_dir, val, preprocess=True, hparams=hparams)


print(train_dataset[0])

Loaded data from /users/santoshp/uiuc/quantumGNN/data/train_set/event000021044.pyg
Preprocessing data
DataBatch(gphi=[10374], phi=[10374], lphi=[10374], lx=[10374], z=[10374], weight=[10374], lz=[10374], region=[10374], cell_count=[10374], ly=[10374], r=[10374], eta=[10374], geta=[10374], y=[37031], hit_id=[10374], leta=[10374], cell_val=[10374], x=[10374], module_index=[10374], track_edges=[2, 9453], radius=[9453], particle_id=[9453], pt=[9453], nhits=[9453], config=[2], event_id=[1], num_nodes=10374, batch=[10374], ptr=[2], edge_index=[2, 37031], truth_map=[9453], dr=[37031], dz=[37031], dphi=[37031])


In [5]:
params = {'batch_size': 1, 'shuffle': True, 'num_workers': 0}
train_loader = DataLoader(train_dataset,**params)  #batches join graphs instead of splitting them therefore more than train set batches will make 1 batch only 
test_loader = DataLoader(test_dataset, **params)
val_loader = DataLoader(val_dataset, **params)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
def make_quantum_mlp(
    input_size: int,
    sizes: List[int],
    hidden_activation: str = "ReLU",
    output_activation: Optional[str] = None,
    n_qubits: int = 2,
    n_qlayers: int = 1,
    device: str = "default.qubit",
    apply_quantum_output: bool = False,
    verbose: bool = True,
    # Classical MLP compatibility args (ignored in quantum MLP)
    layer_norm: bool = False,
    output_layer_norm: bool = False,
    batch_norm: bool = False,
    output_batch_norm: bool = False,
    input_dropout: float = 0.0,
    hidden_dropout: float = 0.0,
    track_running_stats: bool = False,
) -> nn.Module:
    """
    Quantum MLP fully compatible with classical make_mlp interface.
    Original architecture is preserved.
    """
    # ------------------ original quantum MLP code ------------------
    if n_qubits < 1:
        raise ValueError(f"Number of qubits must be at least 1, got {n_qubits}")
    if n_qlayers < 1:
        raise ValueError(f"Number of quantum layers must be at least 1, got {n_qlayers}")
    if not sizes:
        raise ValueError("Sizes list cannot be empty")
    if input_size < 1:
        raise ValueError(f"Input size must be at least 1, got {input_size}")
    if any(size < 1 for size in sizes):
        raise ValueError(f"All sizes must be at least 1, got {sizes}")

    try:
        q_device = qml.device(device, wires=n_qubits)
    except Exception as e:
        raise ValueError(f"Failed to initialize quantum device {device}: {str(e)}")

    def quantum_circuit(inputs, weights):
        inputs = torch.tanh(inputs) * torch.pi
        for i in range(n_qubits):
            if i < inputs.shape[1]:
                qml.RY(inputs[:, i], wires=i)
            else:
                qml.RY(0.0, wires=i)
        for layer in range(n_qlayers):
            for w in range(n_qubits):
                qml.RX(weights[layer, w, 0], wires=w)
                qml.RZ(weights[layer, w, 1], wires=w)
                qml.RY(weights[layer, w, 2], wires=w)
            for w in range(n_qubits - 1):
                qml.CNOT(wires=[w, w + 1])
            if n_qubits >= 2:
                for w in range(n_qubits - 1):
                    qml.CRZ(weights[layer, w, 3], wires=[w, w + 1])
        return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

    n_params_per_qubit = 4 if n_qubits >= 2 else 2
    weight_shapes = {"weights": (n_qlayers, n_qubits, n_params_per_qubit)}

    q_layer_1 = qml.QNode(quantum_circuit, q_device, interface="torch", diff_method="backprop")
    q_layer_1 = qml.qnn.TorchLayer(q_layer_1, weight_shapes)
    q_layer_2 = qml.QNode(quantum_circuit, q_device, interface="torch", diff_method="backprop")
    q_layer_2 = qml.qnn.TorchLayer(q_layer_2, weight_shapes)

    try:
        hidden_activation_fn = getattr(nn, hidden_activation)
    except AttributeError:
        raise ValueError(f"Invalid hidden_activation: {hidden_activation}")
    output_activation_fn = None
    if output_activation:
        try:
            output_activation_fn = getattr(nn, output_activation)
        except AttributeError:
            raise ValueError(f"Invalid output_activation: {output_activation}")

    layers = []
    sizes = [input_size] + sizes
    current_size = input_size

    # Minimal classical input mapping
    if input_size != n_qubits:
        layers.append(nn.Linear(input_size, n_qubits))
        current_size = n_qubits

    # Two quantum layers
    layers.append(q_layer_1)
    layers.append(q_layer_2)
    current_size = n_qubits

    # Minimal classical output mapping
    if current_size != sizes[-1]:
        layers.append(nn.Linear(current_size, sizes[-1]))
        current_size = sizes[-1]
    if output_activation_fn:
        layers.append(output_activation_fn())

    mlp_model = nn.Sequential(*layers)

    # Register quantum parameters explicitly
    for i, q_layer in enumerate([q_layer_1, q_layer_2], 1):
        for name, param in q_layer.named_parameters():
            mlp_model.register_parameter(f"quantum_layer_{i}_{name}", param)

    if verbose:
        print("\n==== Quantum MLP Architecture ====")
        classical_params = 0
        quantum_params = 0
        for i, layer in enumerate(layers):
            params = sum(p.numel() for p in layer.parameters()) if hasattr(layer, 'parameters') else 0
            if isinstance(layer, qml.qnn.TorchLayer):
                quantum_params += params
            else:
                classical_params += params
            print(f"Layer {i}: {layer}, Parameters: {params}")
        print("===================================")
        print("\n==== Quantum MLP Parameters ====")
        total_params = classical_params + quantum_params
        print(f"Total trainable parameters: {total_params}")
        print(f"Quantum parameters: {quantum_params}")
        print(f"Classical parameters: {classical_params}")
        print(f"Circuit: {n_qubits} qubits, {n_qlayers} layers, device={device}")
        print("===================================")
        print("===============Print-Model====================")
        print(mlp_model)
        print("===================================")

    return mlp_model

In [8]:
warnings.filterwarnings("ignore")
class QuantumInteractionGNN2(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.criterion = nn.BCEWithLogitsLoss()

        # Set default hyperparameters
        hparams.setdefault("batchnorm", False)
        hparams.setdefault("output_batch_norm", False)
        hparams.setdefault("edge_output_transform_final_batch_norm", False)
        hparams.setdefault("track_running_stats", False)
        hparams.setdefault("quantum_device", "default.qubit")
        hparams.setdefault("checkpointing", True)
        hparams.setdefault("concat", True)
        hparams.setdefault("in_out_diff_agg", True)
        hparams.setdefault("edge_net_recurrent", True)
        hparams.setdefault("node_net_recurrent", True)

        # Aggregation function
        self.aggr_function = scatter_add if hparams.get("aggr", "sum") == "sum" else scatter_mean

        # Determine input sizes for node and edge networks
        in_node_net = hparams["hidden"] * (
            4 if hparams["concat"] and hparams["in_out_diff_agg"] else
            3 if hparams["concat"] else
            3 if hparams["in_out_diff_agg"] else 2
        )
        in_edge_net = hparams["hidden"] * 6 if hparams["concat"] else hparams["hidden"] * 3

        # Node Encoder (Quantum MLP)
        self.node_encoder = make_quantum_mlp(
            input_size=len(hparams["node_features"]),
            sizes=[hparams["hidden"]] * hparams["n_node_net_layers"],
            hidden_activation=hparams["hidden_activation"],
            output_activation=hparams["output_activation"],
            n_qubits=hparams["n_qubits"],
            n_qlayers=hparams["n_qlayers"],
            device=hparams["quantum_device"],
            verbose=False,
        )

        # Edge Encoder (Quantum MLP)
        edge_input_size = len(hparams["edge_features"]) if hparams.get("edge_features") else 2 * hparams["hidden"]
        self.edge_encoder = make_quantum_mlp(
            input_size=edge_input_size,
            sizes=[hparams["hidden"]] * hparams["n_edge_net_layers"],
            hidden_activation=hparams["hidden_activation"],
            output_activation=hparams["output_activation"],
            n_qubits=hparams["n_qubits"],
            n_qlayers=hparams["n_qlayers"],
            device=hparams["quantum_device"],
            verbose=False,
        )

        # Edge Network (Quantum MLP)
        if hparams["edge_net_recurrent"]:
            self.edge_network = make_quantum_mlp(
                input_size=in_edge_net,
                sizes=[hparams["hidden"]] * hparams["n_edge_net_layers"],
                hidden_activation=hparams["hidden_activation"],
                output_activation=hparams["output_activation"],
                n_qubits=hparams["n_qubits"],
                n_qlayers=hparams["n_qlayers"],
                device=hparams["quantum_device"],
                verbose=False,
            )
        else:
            self.edge_network = nn.ModuleList([
                make_quantum_mlp(
                    input_size=in_edge_net,
                    sizes=[hparams["hidden"]] * hparams["n_edge_net_layers"],
                    hidden_activation=hparams["hidden_activation"],
                    output_activation=hparams["output_activation"],
                    n_qubits=hparams["n_qubits"],
                    n_qlayers=hparams["n_qlayers"],
                    device=hparams["quantum_device"],
                    verbose=False,
                ) for _ in range(hparams["n_graph_iters"])
            ])

        # Node Network (Quantum MLP)
        if hparams["node_net_recurrent"]:
            self.node_network = make_quantum_mlp(
                input_size=in_node_net,
                sizes=[hparams["hidden"]] * hparams["n_node_net_layers"],
                hidden_activation=hparams["hidden_activation"],
                output_activation=hparams["output_activation"],
                n_qubits=hparams["n_qubits"],
                n_qlayers=hparams["n_qlayers"],
                device=hparams["quantum_device"],
                verbose=False,
            )
        else:
            self.node_network = nn.ModuleList([
                make_quantum_mlp(
                    input_size=in_node_net,
                    sizes=[hparams["hidden"]] * hparams["n_node_net_layers"],
                    hidden_activation=hparams["hidden_activation"],
                    output_activation=hparams["output_activation"],
                    n_qubits=hparams["n_qubits"],
                    n_qlayers=hparams["n_qlayers"],
                    device=hparams["quantum_device"],
                    verbose=False,
                ) for _ in range(hparams["n_graph_iters"])
            ])

        # Edge Decoder (Quantum MLP)
        self.edge_decoder = make_quantum_mlp(
            input_size=hparams["hidden"],
            sizes=[hparams["hidden"]] * hparams["n_edge_decoder_layers"],
            hidden_activation=hparams["hidden_activation"],
            output_activation=None,
            n_qubits=hparams["n_qubits"],
            n_qlayers=hparams["n_qlayers"],
            device=hparams["quantum_device"],
            verbose=False,
        )

        # Edge output transform (classical linear for logits)
        self.edge_output_transform = nn.Linear(hparams["hidden"], 1)

    ###############################
    # Training/validation/testing hooks
    def training_step(self, batch, batch_idx):
        logits = self(batch)
        y = batch.y.float()
        loss = self.criterion(logits, y)

        # Metrics
        preds = torch.sigmoid(logits)
        acc = ((preds > 0.5) == y).float().mean()
        try:
            auc = roc_auc_score(y.cpu().numpy(), preds.detach().cpu().numpy())
        except ValueError:
            auc = torch.tensor(0.0)  # if only one class in batch

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_auc", auc, on_step=True, on_epoch=True, prog_bar=True)

        print(f" Training: Loss: {loss.item():.4f} | Acc: {acc.item():.4f} | AUC: {auc:.4f}")

        return loss

    def validation_step(self, batch, batch_idx):
        logits = self(batch)
        y = batch.y.float()
        loss = self.criterion(logits, y)

        # Metrics
        preds = torch.sigmoid(logits)
        acc = ((preds > 0.5) == y).float().mean()
        try:
            auc = roc_auc_score(y.cpu().numpy(), preds.detach().cpu().numpy())
        except ValueError:
            auc = torch.tensor(0.0)

        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        self.log("val_auc", auc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        logits = self(batch)
        y = batch.y.float()
        loss = self.criterion(logits, y)

        # Metrics
        preds = torch.sigmoid(logits)
        acc = ((preds > 0.5) == y).float().mean()
        try:
            auc = roc_auc_score(y.cpu().numpy(), preds.detach().cpu().numpy())
        except ValueError:
            auc = torch.tensor(0.0)

        self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("test_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        self.log("test_auc", auc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.get("lr", 0.0001))

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4)

    def test_dataloader(self):
        return DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

    ###############################
    # Forward and message passing
    def forward(self, batch):
        x = torch.stack([batch[feature] for feature in self.hparams["node_features"]], dim=-1).float()
        edge_attr = torch.stack([batch[feature] for feature in self.hparams["edge_features"]], dim=-1).float() if self.hparams.get("edge_features") else None
        src, dst = batch.edge_index
        x.requires_grad = True
        if edge_attr is not None:
            edge_attr.requires_grad = True

        # Node and edge encoding
        if self.hparams["checkpointing"]:
            x = checkpoint(self.node_encoder, x, use_reentrant=False)
            e = checkpoint(self.edge_encoder, edge_attr) if edge_attr is not None else checkpoint(self.edge_encoder, torch.cat([x[src], x[dst]], dim=-1))
        else:
            x = self.node_encoder(x)
            e = self.edge_encoder(edge_attr if edge_attr is not None else torch.cat([x[src], x[dst]], dim=-1))

        # Keep original for concat
        if self.hparams["concat"]:
            input_x, input_e = x, e

        outputs = []
        for i in range(self.hparams["n_graph_iters"]):
            if self.hparams["checkpointing"] and self.hparams["concat"]:
                x = checkpoint(self.concat, x, input_x)
                e = checkpoint(self.concat, e, input_e)
            x, e, out = self.message_step(x, e, src, dst, i)
            outputs.append(out)

        return outputs[-1].squeeze(-1)

    def message_step(self, x, e, src, dst, i=None):
        edge_inputs = torch.cat([e, x[src], x[dst]], dim=-1)
        if self.hparams["edge_net_recurrent"]:
            e_updated = self.edge_network(edge_inputs)
        else:
            e_updated = self.edge_network[i](edge_inputs)

        edge_messages_src = self.aggr_function(e_updated, dst, dim=0, dim_size=x.shape[0])
        edge_messages_dst = self.aggr_function(e_updated, src, dim=0, dim_size=x.shape[0])

        if self.hparams["in_out_diff_agg"]:
            node_inputs = torch.cat([edge_messages_src, edge_messages_dst, x], dim=-1)
        else:
            edge_messages = edge_messages_src + edge_messages_dst
            node_inputs = torch.cat([edge_messages, x], dim=-1)

        if self.hparams["node_net_recurrent"]:
            x_updated = self.node_network(node_inputs)
        else:
            x_updated = self.node_network[i](node_inputs)

        edge_logits = self.edge_output_transform(self.edge_decoder(e_updated))
        return x_updated, e_updated, edge_logits

    def concat(self, x, y):
        return torch.cat([x, y], dim=-1)


In [9]:
model = QuantumInteractionGNN2(hparams)
trainer = pl.Trainer(
    max_epochs=2,
    log_every_n_steps=1,   # <-- show logs every step
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
)
trainer.fit(model)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
2026-01-07 03:43:01.001547: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name                  ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ criterion             │ BCEWithLogitsLoss │      0 │ train │     0 │
│ 1 │ node_encoder          │ Sequential        │     72 │ train │     0 │
│ 2 │ edge_encoder          │ Sequential        │     72 │ train │     0 │
│ 3 │ edge_network          │ Sequential        │    258 │ train │     0 │
│ 4 │ node_network          │ Sequential        │    194 │ train │     0 │
│ 5 │ edge_decoder          │ Sequential        │     98 │ train │     0 │
│ 6 │ edge_output_transform │ Linear            │     17 │ train │     0 │
└───┴───────────────────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 711                                                                                              
Non-trainable params: 0                                                                                            
Total params: 711                                                                                                  
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 31                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Training: Loss: 0.7238 | Acc: 0.2612 | AUC: 0.6100

Training: Loss: 0.7278 | Acc: 0.2215 | AUC: 0.6397

...

Training: Loss: 0.7160 | Acc: 0.2612 | AUC: 0.6304

Training: Loss: 0.7194 | Acc: 0.2154 | AUC: 0.6573

Training: Loss: 0.7168 | Acc: 0.2399 | AUC: 0.6542

Training: Loss: 0.7177 | Acc: 0.2215 | AUC: 0.6531

`Trainer.fit` stopped: `max_epochs=2` reached.


In [10]:
model.eval()

QuantumInteractionGNN2(
  (criterion): BCEWithLogitsLoss()
  (node_encoder): Sequential(
    (0): Linear(in_features=3, out_features=2, bias=True)
    (1): <Quantum Torch Layer: func=quantum_circuit>
    (2): <Quantum Torch Layer: func=quantum_circuit>
    (3): Linear(in_features=2, out_features=16, bias=True)
    (4): ReLU()
  )
  (edge_encoder): Sequential(
    (0): Linear(in_features=3, out_features=2, bias=True)
    (1): <Quantum Torch Layer: func=quantum_circuit>
    (2): <Quantum Torch Layer: func=quantum_circuit>
    (3): Linear(in_features=2, out_features=16, bias=True)
    (4): ReLU()
  )
  (edge_network): Sequential(
    (0): Linear(in_features=96, out_features=2, bias=True)
    (1): <Quantum Torch Layer: func=quantum_circuit>
    (2): <Quantum Torch Layer: func=quantum_circuit>
    (3): Linear(in_features=2, out_features=16, bias=True)
    (4): ReLU()
  )
  (node_network): Sequential(
    (0): Linear(in_features=64, out_features=2, bias=True)
    (1): <Quantum Torch Layer: f

In [11]:
with torch.no_grad():
    for batch in val_loader:
        edge_scores = model(batch)
        print(edge_scores)
        print(edge_scores.size(0))
        positive_scores = edge_scores[edge_scores > 0]
        print(positive_scores)
        print(positive_scores.size(0))

Loaded data from /users/santoshp/uiuc/quantumGNN/data/val_set/event000021283.pyg
Preprocessing data
tensor([0.0876, 0.0861, 0.0859,  ..., 0.0879, 0.0861, 0.0866])
36160
tensor([0.0876, 0.0861, 0.0859,  ..., 0.0879, 0.0861, 0.0866])
36160
Loaded data from /users/santoshp/uiuc/quantumGNN/data/val_set/event000021346.pyg
Preprocessing data
tensor([0.0867, 0.0868, 0.0860,  ..., 0.0859, 0.0854, 0.0864])
35598
tensor([0.0867, 0.0868, 0.0860,  ..., 0.0859, 0.0854, 0.0864])
35598
Loaded data from /users/santoshp/uiuc/quantumGNN/data/val_set/event000021334.pyg
Preprocessing data
tensor([0.0867, 0.0865, 0.0858,  ..., 0.0831, 0.0818, 0.0856])
42860
tensor([0.0867, 0.0865, 0.0858,  ..., 0.0831, 0.0818, 0.0856])
42860
Loaded data from /users/santoshp/uiuc/quantumGNN/data/val_set/event000021349.pyg
Preprocessing data
tensor([0.0852, 0.0861, 0.0867,  ..., 0.0865, 0.0869, 0.0844])
42357
tensor([0.0852, 0.0861, 0.0867,  ..., 0.0865, 0.0869, 0.0844])
42357
Loaded data from /users/santoshp/uiuc/quantumGNN